# Write C++ wrapper for ghostbasil

`ml cmake/3.11.1 gcc/12.1.0 julia/1.8.4`

1. We will be using [CxxWrap.jl](https://github.com/JuliaInterop/CxxWrap.jl) to call C++ code within Julia. 
2. First, we must create a `libcxxwrap_julia_jll` JLL library, following [its github instructions](https://github.com/JuliaInterop/libcxxwrap-julia). 
3. Once it's done, I got a folder `/home/groups/sabatti/.julia/dev/libcxxwrap_julia_jll/override`

## Hello world example

Here is my `hello.cpp` file:
```cpp
#include <string>
#include "jlcxx/jlcxx.hpp"

std::string greet()
{
   return "hello, world";
}

JLCXX_MODULE define_julia_module(jlcxx::Module& mod)
{
  mod.method("greet", &greet);
}
```

And here is the `CMakeLists.txt` file:

```cmake
project(HelloWorld)

cmake_minimum_required(VERSION 3.5)
set(CMAKE_MACOSX_RPATH 1)
set(CMAKE_LIBRARY_OUTPUT_DIRECTORY "${CMAKE_BINARY_DIR}/lib")

find_package(JlCxx)
get_target_property(JlCxx_location JlCxx::cxxwrap_julia LOCATION)
get_filename_component(JlCxx_location ${JlCxx_location} DIRECTORY)
set(CMAKE_INSTALL_RPATH "${CMAKE_INSTALL_PREFIX}/lib;${JlCxx_location}")

message(STATUS "Found JlCxx at ${JlCxx_location}")

add_library(hello SHARED hello.cpp)

target_link_libraries(hello JlCxx::cxxwrap_julia)

install(TARGETS
  hello
LIBRARY DESTINATION lib
ARCHIVE DESTINATION lib
RUNTIME DESTINATION lib)
```

Given these 2 files, we build the shared library (`.io`) file as
```
mkdir build && cd build
cmake -DCMAKE_BUILD_TYPE=Release -DCMAKE_PREFIX_PATH=/home/groups/sabatti/.julia/dev/libcxxwrap_julia_jll/override /home/users/bbchu/ghostbasilwrap
cmake --build . --config Release
```
Finally, we can call hello world in Julia:

In [1]:
# Load the module and generate the functions
module CppHello
using CxxWrap
@wrapmodule(joinpath("/home/users/bbchu/ghostbasilwrap/build/lib/libhello"))

function __init__()
    @initcxx
end
end

# Call greet and show the result
@show CppHello.greet()

CppHello.greet() = "hello, world"


"hello, world"